In [ ]:
import pandas as pd
import numpy as np

import datetime

import json
import re
import string

import collections
from itertools import chain

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from nltk.stem.porter import PorterStemmer

In [ ]:
data = pd.read_csv('data_processed.csv', lineterminator='\n')

In [ ]:
features = data[['AccountTweets', 'AccountFollowers', 'AccountFollowing', 'rt_count', 'urls_count',
       'capital_count', 'tweet_length', 'punctuation_count', 'hashtag_count', 
       'emoji_count', 'laugh_count', 'swear_count',
       'diff_words_count', 'abbr_count', 'count_13_18_pos',
       'count_13_18_neg', 'count_19_22_pos', 'count_19_22_neg',
       'count_23_29_pos', 'count_23_29_neg', 'count_30_pos', 'count_30_neg',
       'nnp', 'nn', 'rb', 'in', 'fw', 'nnps', 'vbp', 'cd', 'vbd', 'md', 'vb',
       'vbg', 'vbz', 'rbr', 'vbn', 'jjs', 'cc',
       'site', 'creation_date', 'literal_emojis', 
                     'emoji0',
 'emoji1',
 'emoji2',
 'emoji3',
 'emoji4',
 'emoji5',
 'emoji6',
 'emoji7',
 'emoji8',
 'emoji9',
 'emoji10',
 'emoji11',
 'emoji12',
 'emoji13',
 'emoji14',
 'emoji15',
 'emoji16',
 'emoji17',
 'emoji18',
 'emoji19',
 'emoji20',
 'emoji21',
 'emoji22',
 'emoji23',
 'emoji24',
 'emoji25',
 'emoji26',
 'emoji27',
 'emoji28',
 'emoji29',
 'emoji30',
 'emoji31',
 'emoji32',
 'emoji33',
 'emoji34',
 'emoji35',
 'emoji36',
 'emoji37',
 'emoji38',
 'emoji39',
 'emoji40',
 'emoji41',
 'emoji42',
 'emoji43',
 'emoji44',
 'emoji45',
 'emoji46',
 'emoji47',
 'emoji48',
 'emoji49',
 'emoji50',
 'emoji51',
 'emoji52',
 'emoji53',
 'emoji54',
 'emoji55',
 'emoji56',
 'emoji57',
 'emoji58',
 'emoji59',
 'emoji60',
 'emoji61',
 'emoji62',
 'emoji63',
 'emoji64',
 'emoji65',
 'emoji66',
 'emoji67',
 'emoji68',
 'emoji69',
 'emoji70',
 'emoji71',
 'emoji72',
 'emoji73',
 'emoji74',
 'emoji75',
 'emoji76',
 'emoji77',
 'emoji78',
 'emoji79',
 'emoji80',
 'emoji81',
 'emoji82',
 'emoji83',
 'emoji84',
 'emoji85',
 'emoji86',
 'emoji87',
 'emoji88',
 'emoji89',
 'emoji90',
 'emoji91',
 'emoji92',
 'emoji93',
 'emoji94',
 'emoji95',
 'emoji96',
 'emoji97',
 'emoji98',
 'emoji99']]
labels = data['Class']

In [ ]:
normalized_features = preprocessing.normalize(features)
normalized_features_df = pd.DataFrame(normalized_features)
normalized_features_df.columns = features.columns

In [ ]:
labels0 = []
for i in data['Class']:
    if i == 0:
        labels0.append(1)
    else:
        labels0.append(0)

In [ ]:
labels1 = []
for i in data['Class']:
    if i == 1:
        labels1.append(1)
    else:
        labels1.append(0)

In [ ]:
labels2 = []
for i in data['Class']:
    if i == 2:
        labels2.append(1)
    else:
        labels2.append(0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(normalized_features_df, labels, test_size=0.2)

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth = 7, learning_rate=0.05)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print(metrics.classification_report(y_test,y_pred))

In [ ]:
scorings = ['accuracy', 'f1_macro', 'f1_micro', 'precision', 'recall']
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth = 7, learning_rate=0.05)
cross_validate(model, X_train, y_train, scoring=scorings, cv=5)

In [ ]:
matrix = confusion_matrix(y_test, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

In [ ]:
import xgboost as xgb
xgb.plot_importance(model, max_num_features = 22, importance_type='gain')

In [ ]:
def computeTFIDFVector(dct_igm, review):
    # Create a list of unique words
    wordDict = sorted(dct_igm.keys())
    tfidfVector = [0.0] * len(wordDict)
    
  # For each unique word, if it is in the review, store its TF-IDF value.
    for i, word in enumerate(wordDict):
        if word in review:
            tfidfVector[i] = review[word]
    return tfidfVector

def computeCountDict(list_tf):
    
    """ Returns a dictionary whose keys are all the unique words in
    the dataset and whose values count the number of reviews in which
    the word appears.
    """
    countDict = {}
    # Run through each review's tf dictionary and increment countDict's (word, doc) pair
    for review in list_tf:
        for word in review:
            if word in countDict:
                countDict[word] += 1
            else:
                countDict[word] = 1
    return countDict

def computeReviewTFDict(review):
    """ Returns a tf dictionary for each review whose keys are all
    the unique words in the review and whose values are their
    corresponding tf.
    """
    # Counts the number of times the word appears in review
    reviewTFDict = {}
    for word in review:
        if word in reviewTFDict:
            reviewTFDict[word] += 1
        else:
            reviewTFDict[word] = 1
    # Computes tf for each word
    for word in reviewTFDict:
        reviewTFDict[word] = reviewTFDict[word] / len(review)
    return reviewTFDict

def map_dict(dct_igm, review):
    reviewTFDict = {}
    output = list(map(dct_igm.get, review.keys()))
    output = [0 if v is None else v for v in output]
    output = [v/np.log(len(output)) for v in output]
    
    reviewTFDict = dict(zip(review.keys(), output))
    return reviewTFDict

In [ ]:
### IGM

data_new = data[['Volgendaccount', 'Class']]

following_ids = data_new['Volgendaccount']
following_ids0 = data_new[data_new['Class']==0]['Volgendaccount']
following_ids1 = data_new[data_new['Class']==1]['Volgendaccount']
following_ids2 = data_new[data_new['Class']==2]['Volgendaccount']

data0 = [
    [(word.replace(",", "")
          .replace(".", "")
          .replace("(", "")
          .replace(")", "")
          .replace("]", "")
          .replace("[", ""))
    for word in row.lower().split()]
    for row in following_ids0]

data1 = [
    [(word.replace(",", "")
          .replace(".", "")
          .replace("(", "")
          .replace(")", "")
          .replace("]", "")
          .replace("[", ""))
    for word in row.lower().split()]
    for row in following_ids1]

data2 = [
    [(word.replace(",", "")
          .replace(".", "")
          .replace("(", "")
          .replace(")", "")
          .replace("]", "")
          .replace("[", ""))
    for word in row.lower().split()]
    for row in following_ids2]

data_all = [
    [(word.replace(",", "")
          .replace(".", "")
          .replace("(", "")
          .replace(")", "")
          .replace("]", "")
          .replace("[", ""))
    for word in row.lower().split()]
    for row in following_ids]

list_tf = []
list_tf0 = []
list_tf1 = []
list_tf2 = []
for i in data_all:
    list_tf.append(computeReviewTFDict(i))
for i in data0:
    list_tf0.append(computeReviewTFDict(i))
for i in data1:
    list_tf1.append(computeReviewTFDict(i))
for i in data2:
    list_tf2.append(computeReviewTFDict(i))
    
countDict0 = computeCountDict(list_tf0)
countDict1 = computeCountDict(list_tf1)
countDict2 = computeCountDict(list_tf2)
    
df_counts0 = pd.DataFrame.from_dict(countDict0, orient='index')
df_counts1 = pd.DataFrame.from_dict(countDict1, orient='index')
df_counts2 = pd.DataFrame.from_dict(countDict2, orient='index')

df = pd.concat([df_counts0, df_counts1, df_counts2], axis=1, sort=False)

df = df.fillna(0)
df.columns = ['counts0', 'counts1', 'counts2']
df['sum'] = df.sum(axis=1)
df['max'] = df[['counts0', 'counts1', 'counts2']].max(axis=1)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
min_following = 1 # This was 5 in the study

In [ ]:
df = df[df['counts0'] >= min_following]
df = df[df['counts1'] >= min_following]
df = df[df['counts2'] >= min_following]

In [ ]:
df.shape

In [ ]:
df['igm'] = df['max']/df['sum']

In [ ]:
df[df['counts1'] == df['max']].sort_values('igm', ascending=False)

In [ ]:
# 163730859 => @JackJ

# 308297673 => @KianLawley

# 405728790 => @camerondallas

# 216430965 => @Caspar_Lee

# 3105826730 => @DemetriusHarmon

In [ ]:
# 34900333 => @jennajameson

# 19339713 => @timlovejoy

# 15901190 => @trent_reznor

# 91208190 => @MelKiperESPN

# 308981378 => @JohnHartson10

In [ ]:
# 112047805 => @brithume

# 223970563 => @TeamCavuto

# 15220768 => @DavidCornDC

# 232901331 => @dbongino

# 56561449 => @JesseBWatters

In [ ]:
dct_igm = dict(zip(df.index,df['igm']))
for i in dct_igm:
    if i == 1.0:
        i = 0
        
users_mapped = []
for i in list_tf:
    users_mapped.append(map_dict(dct_igm, i))
    
igm_result = [computeTFIDFVector(dct_igm, account_id) for account_id in users_mapped]

In [ ]:
igm_array = np.array(igm_result)
igm_df = pd.DataFrame(igm_array)
normalized_igm = preprocessing.normalize(igm_df)
normalized_igm_df = pd.DataFrame(normalized_igm)

In [ ]:
columns_igm = []
for i in range(len(normalized_igm_df.columns)):
    columns_igm.append('igm'+str(i))

In [ ]:
normalized_igm_df.columns = columns_igm

In [ ]:
igm_features_normalized=pd.concat([normalized_igm_df,normalized_features_df], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(igm_features_normalized, labels, test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print(metrics.classification_report(y_test,y_pred))

In [ ]:
matrix = confusion_matrix(y_test, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

In [ ]:
scorings = ['accuracy', 'f1_macro', 'f1_micro', 'precision', 'recall']
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False, random_state=0)
cross_validate(model, X_train, y_train, scoring=scorings, cv=5)

In [ ]:
all_tweets = []
for tweets in data['Tweets']:
    rt = 0
    tweets_user = []
    for i in range(len(tweets.split("\'"))):
        tweet_fragment = tweets.split("\'")[i]
        if tweet_fragment.count('RT') != 0:
            x = 1
        else:
            tweets_user.append(tweet_fragment)
    all_tweets.append(' '.join(tweets_user))
    
    if len(all_tweets) % 1000 == 0:
        print(len(all_tweets))

In [ ]:
clean_tweets = []

for tweet in all_tweets:
    
    tweet_no_link = re.sub(r'http\S+', '', tweet)
    tweet_stripped = tweet_no_link.lower().translate(str.maketrans('', '', '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~')).strip()
    clean_tweet = re.sub(' +', ' ', tweet_stripped)
    
    clean_tweets.append(clean_tweet)
    
    if len(clean_tweets) % 1000 == 0:
        print(len(clean_tweets))

In [ ]:
final_tweets = []
for tweet in clean_tweets:
    no_handle = re.sub('(?:\s)@[^, ]*', '', tweet)
    no_hashtag = re.sub('(?:\s)#[^, ]*', '', no_handle)
    final_tweets.append(no_hashtag)
    
    if len(final_tweets) % 1000 == 0:
        print(len(final_tweets))

In [ ]:
data['sage_tweets'] = final_tweets

In [ ]:
stemmer = PorterStemmer()
tweets_stemmed = []
for tweet in data['sage_tweets']:
    tokenAux=""
    textAux=""
    tokens = tweet.split()
    for token in tokens:
        tokenAux = token
        tokenAux = stemmer.stem(token)    
        textAux = textAux + " "+ tokenAux
    tweets_stemmed.append(textAux)
    
    if len(tweets_stemmed) % 1000 == 0:
        print(len(tweets_stemmed))

In [ ]:
data['tweets_stemmed'] = tweets_stemmed

In [ ]:
import os
import re
from collections import Counter

# import en_core_web_sm

import math
from numpy.linalg import norm
import copy
from scipy.optimize import minimize

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string

from itertools import chain

def adj_noun_merger(doc):
    offset = 0
    while offset < len(doc) - 3:
        if doc[offset].pos_ in ["ADJ", "NOUN"] and doc[offset+1].pos_ == "NOUN":
            start = doc[offset].i
            if doc[offset+2].pos_ == 'NOUN':
                if doc[offset+3].pos_ == 'NOUN': end = doc[offset+3].i
                else: end = doc[offset+2].i 
            else: end = doc[offset+1].i
            with doc.retokenize() as retokenizer:
                retokenizer.merge(doc[start:end+1], attrs={"POS" : "NOUN"}) 
        offset += 1
        
def get_counters(left, right):
    from collections import Counter
    base_dict = {token : 1 for token in list(set(left + right))}
    left_counts, right_counts = Counter(left), Counter(right)
    
    left_dict, right_dict = base_dict.copy(), base_dict.copy()
    left_dict.update(left_counts)
    right_dict.update(right_counts)
    
    return left_dict, right_dict

def topK(beta,vocab,K=10):
    return [vocab[idx] for idx in (-beta).argsort()[:K]]

# import SAGE

def get_keywords(eta, vocab, min_len = 2, remove = []):
    keywords = [i[0] for i in topK(eta,vocab,len(vocab)) if i[1] == 'NOUN' and len(i[0].split(' ')) >= min_len and not any(c in string.digits for c in i[0])]
    for word in remove:
        try: keywords.remove(word)
        except ValueError: print(f'{word} not in keywords')
    return keywords

class DeltaIterator:
    def __init__(s,max_its=100,thresh=1e-4,debug=False):
        s.thresh = thresh
        s.max_its = max_its
        s.prev = None
        s.done = False
        s.its = 0
        s.debug = debug

    def update(s,x):
        if s.prev is not None:
            change = norm(x - s.prev) / (1e-6+norm(x))
            if s.debug: print(s.its,'/',s.max_its,change)
            if change < s.thresh: s.done = True
        s.its += 1
        if s.its > s.max_its: s.done = True
        s.prev = copy.copy(x)

def estimate(ecounts,eq_m,max_its=25):
    if len(ecounts.shape)==1:
        ecounts = np.reshape(ecounts,(-1,1))
    [W,K] = ecounts.shape
    eta = np.zeros(W)
    eq_inv_tau = np.ones(W)
    exp_eq_m = np.exp(eq_m)
    max_inv_tau = 1e5
    it = DeltaIterator(debug=False,max_its=max_its,thresh=1e-4)
    while not(it.done):
        fLogNormal = lambda x : fLogNormalAux(x,ecounts,exp_eq_m,eq_inv_tau)
        gLogNormal = lambda x : gLogNormalAux(x,ecounts,exp_eq_m,eq_inv_tau)
        min_out = minimize(fLogNormal,eta,method='L-BFGS-B',jac=gLogNormal,options={'disp':False})
        #TODO:
        #hpLogNormal = lambda x : hpLogNormalAux(x,ecounts,exp_eq_m,eq_inv_tau)
        #min_out = minimize(fLogNormal,eta,method='Newton-CG',jac=gLogNormal,options={'disp':True})
        eta = min_out.x
        eq_inv_tau = 1/(eta**2)
        eq_inv_tau[eq_inv_tau > max_inv_tau] = max_inv_tau
        it.update(eta)
    return(eta)

def fLogNormalAux(eta,ecounts,exp_eq_m,eq_inv_tau):
    C = ecounts.sum(axis=0)
    [W,K] = ecounts.shape
    denom = np.tile(np.exp(eta),(K,1)).dot(exp_eq_m.T)
    out = -(eta.T.dot(ecounts).sum(axis=0) - C * np.log(denom.sum(axis=0)) - 0.5 * eq_inv_tau.T.dot(eta ** 2))
    return(out[0])
           
def gLogNormalAux(eta,ecounts,exp_eq_m,eq_inv_tau):
    C = ecounts.sum(axis=0)
    [W,K] = ecounts.shape
    denom = np.tile(np.exp(eta),(K,1)) * exp_eq_m
    denom_norm = (denom.T / denom.sum(axis=1))
    beta = C * denom_norm / (C + 1e-10)
    g = -(ecounts.sum(axis=1) - beta.dot(C) - eq_inv_tau * eta)
    return(g)

In [ ]:
bio_teenagers = data[data['Class'] == 0]['tweets_stemmed']
bio_students = data[data['Class'] == 1]['tweets_stemmed']
bio_adults = data[data['Class'] == 2]['tweets_stemmed']

In [ ]:
bio_teenagers = bio_teenagers.str.cat(sep=' ')
bio_students = bio_students.str.cat(sep=' ')
bio_adults = bio_adults.str.cat(sep=' ')

In [ ]:
bio_random_teenagers = bio_students + bio_adults
bio_random_students = bio_teenagers + bio_adults
bio_random_adults = bio_students + bio_teenagers

In [ ]:
tokens_teenagers = word_tokenize(bio_teenagers)
tokens_students = word_tokenize(bio_students)
tokens_adults = word_tokenize(bio_adults)

tokens_random_teenagers = word_tokenize(bio_random_teenagers)
tokens_random_students = word_tokenize(bio_random_students)
tokens_random_adults = word_tokenize(bio_random_adults)

teenage_counter, teenage_random_counter = get_counters(tokens_teenagers, tokens_random_teenagers)
vocab_teenage = [word for word,count in Counter(teenage_counter).most_common(2000)]

students_counter, students_random_counter = get_counters(tokens_students, tokens_random_students)
vocab_students = [word for word,count in Counter(students_counter).most_common(2000)]

adults_counter, adults_random_counter = get_counters(tokens_adults, tokens_random_adults)
vocab_adults = [word for word,count in Counter(adults_counter).most_common(2000)]

In [ ]:
x_teenagers = np.array([teenage_counter[word] for word in vocab_teenage])
x_students = np.array([students_counter[word] for word in vocab_students])
x_adults = np.array([adults_counter[word] for word in vocab_adults])

x_random_teenagers = np.array([teenage_random_counter[word] for word in vocab_teenage]) + 1.
x_random_students = np.array([students_random_counter[word] for word in vocab_students]) + 1.
x_random_adults = np.array([adults_random_counter[word] for word in vocab_adults]) + 1.

mu_teenage = np.log(x_random_teenagers) - np.log(x_random_teenagers.sum())
mu_students = np.log(x_random_students) - np.log(x_random_students.sum())
mu_adults = np.log(x_random_adults) - np.log(x_random_adults.sum())

In [ ]:
eta_teenage = estimate(x_teenagers,mu_teenage)
eta_students = estimate(x_students,mu_students)
eta_adults = estimate(x_adults,mu_adults)

In [ ]:
dct_teenage= dict(zip(vocab_teenage,eta_teenage))
dct_students= dict(zip(vocab_students,eta_students))
dct_adults= dict(zip(vocab_adults,eta_adults))

In [ ]:
all_words = []
for i in data['tweets_stemmed']:
# for i in data['clean_tweets']:
    tokens_user = word_tokenize(str(i))
    word_dct = {}
    for word in tokens_user:
        word_dct[word] = 0
    all_words.append(word_dct)

In [ ]:
words_mapped_teenage = []
for i in all_words:
    words_mapped_teenage.append(map_dict(dct_teenage, i))
    
words_mapped_students = []
for i in all_words:
    words_mapped_students.append(map_dict(dct_students, i))
    
words_mapped_adults = []
for i in all_words:
    words_mapped_adults.append(map_dict(dct_adults, i))

In [ ]:
sage_result_teenage = [computeTFIDFVector(dct_teenage, user) for user in words_mapped_teenage]
sage_result_students = [computeTFIDFVector(dct_students, user) for user in words_mapped_students]
sage_result_adults = [computeTFIDFVector(dct_adults, user) for user in words_mapped_adults]

In [ ]:
array_teenage = np.array(sage_result_teenage)
array_students = np.array(sage_result_students)
array_adults = np.array(sage_result_adults)

In [ ]:
combined = np.column_stack([array_teenage, array_students, array_adults])

In [ ]:
df_sage = pd.DataFrame(combined)
indices_to_keep = ~df_sage.isin([np.nan, np.inf, -np.inf]).any(1)
df_sage_clean = df_sage[indices_to_keep].astype(np.float64)
normalized_sage = preprocessing.normalize(df_sage_clean)
normalized_sage_df = pd.DataFrame(normalized_sage)

In [ ]:
labels_clean = pd.DataFrame(labels).reset_index()[indices_to_keep]['Class']

In [ ]:
igm_features_normalized_kept = igm_features_normalized[indices_to_keep]

In [ ]:
igm_features_normalized_kept.index = normalized_sage_df.index 
normalized_sage_df.reset_index(drop=True, inplace=True)
all_features=pd.concat([normalized_sage_df,igm_features_normalized_kept], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_features, labels_clean, test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False, random_state=0)# , 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print(metrics.classification_report(y_test,y_pred))

In [ ]:
matrix = confusion_matrix(y_test, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

In [ ]:
scorings = ['accuracy', 'f1_macro', 'f1_micro', 'precision', 'recall']
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False, random_state=0)
cross_validate(model, X_train, y_train, scoring=scorings, cv=5)

In [ ]:
all_features_new = all_features

In [ ]:
all_features_new['labels'] = labels_clean

In [ ]:
# f1_scores = []
# for i in range(1000, 9500, 500):
#     sample = all_features_new.sample(i)
    
# #     X_train, X_test, y_train, y_test = train_test_split(sample.iloc[:,:(len(sample.columns)-1)], sample['labels'], test_size=0.2)
    

#     model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
#                        intercept_scaling=1, l1_ratio=None, max_iter=1000,
#                        multi_class='auto', n_jobs=None, penalty='l2',
#                        solver='lbfgs', tol=0.0001, verbose=0,
#                        warm_start=False, random_state=0)# , 
    
#     scores = cross_val_score(model, sample.iloc[:,:(len(sample.columns)-1)], sample['labels'], cv=5, scoring='f1_macro')
#     f1_scores.append(scores.mean())
# #     y_pred = model.predict(X_test)
# #     conf_mat = confusion_matrix(y_test, y_pred)
# #     f1_scores.append(metrics.f1_score(y_test,y_pred, average='macro'))

In [ ]:
f1_scores

In [ ]:
# 0.7065050863020835,
#  0.7277652530808036,
#  0.7433320330531445,
#  0.7428359447186145,
#  0.7348988415913787,
#  0.7445765332468621,
#  0.7561212310164757,
#  0.7486918369390481,
#  0.7563914236012406,
#  0.7527007518167743,
#  0.7574730386507367,
#  0.7585046772120647,
#  0.7573845049163584,
#  0.7626483913823713,
#  0.7676319321924464,
#  0.764989555893724,
#  0.7679941525589526

In [ ]:
topK(eta_adults, vocab_adults, K=50)

In [ ]:
topK(eta_students, vocab_students, K=50)

In [ ]:
topK(eta_teenage, vocab_teenage, K=50)